In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 

In [2]:
tf.__version__

'2.19.0'

In [3]:
pf=pd.read_csv("AmericanExpressSheet.csv")

In [4]:
x= pf.iloc[:,0:-1].values
y= pf.iloc[:,-1].values

In [5]:
print(x)

[[553 'Delhi' 'Female' ... 4 1 274150]
 [447 'Bengaluru' 'Male' ... 4 1 519360]
 [501 'Delhi' 'Female' ... 4 1 545501]
 ...
 [627 'Mumbai' 'Female' ... 4 0 494067]
 [600 'Bengaluru' 'Female' ... 2 1 109375]
 [553 'Delhi' 'Male' ... 4 1 180031]]


In [6]:
print(y)

[0 0 0 ... 0 1 0]


In [7]:
#To find the NaN values
pf.isnull().values.any()
pf[pf.isnull().any(axis=1)]

,Credit Score,Geography,Gender,Age,Customer Since,Current Account,Num of products,UPI Enabled,Estimated Yearly Income,Closed


Gender Column Encoding


In [8]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
x[:,2]= label_encoder.fit_transform(x[:,2])

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
x=np.array(ct.fit_transform(x))

In [10]:
print(x)

[[0.0 1.0 0.0 ... 4 1 274150]
 [1.0 0.0 0.0 ... 4 1 519360]
 [0.0 1.0 0.0 ... 4 1 545501]
 ...
 [0.0 0.0 1.0 ... 4 0 494067]
 [1.0 0.0 0.0 ... 2 1 109375]
 [0.0 1.0 0.0 ... 4 1 180031]]


In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

ANN


Initialization

In [12]:
ann= tf.keras.models.Sequential()

Adding input layer and first hidden layer

In [13]:
ann.add(tf.keras.layers.Dense(units=5, activation='relu'))

Adding second hidden layer


In [14]:
ann.add(tf.keras.layers.Dense(units=5, activation='relu'))

Adding output layer

In [15]:
ann.add(tf.keras.layers.Dense(units=5, activation='sigmoid'))

ANN Training

Compiling ANN

In [16]:
ann.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

Training on training dataset

In [18]:
ann.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
print(type(x_train), x_train.dtype)
print(type(y_train), y_train.dtype)


<class 'numpy.ndarray'> object
<class 'numpy.ndarray'> int64


In [19]:
history=ann.fit(x_train,y_train,batch_size=32, epochs=55,validation_split=0.2)

ValueError: Invalid dtype: object